In [426]:
import os
import pandas as pd
import numpy as np
import pyspark 

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql import functions as F 
from pyspark.sql.types import StringType
from pyspark.sql import dataframe

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline
from sklearn.cluster import KMeans

In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
SparkContext.setSystemProperty('spark.executor.memory', '3g')
SparkContext.setSystemProperty('spark.driver.memory', '3g')

In [6]:
spark._sc.getConf().getAll()

[('spark.executor.memory', '3g'),
 ('spark.driver.memory', '3g'),
 ('spark.app.id', 'local-1582917125534'),
 ('spark.driver.port', '51614'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '192.168.1.82'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [7]:
# show all variables
pd.options.display.max_columns = None 

## 1. Onehot_encode for Outpatient

In [172]:
def df_dummies(df, col_lst):
    
    new_df = pd.get_dummies(df[col_lst])
    new_df['DESYNPUF_ID'] = df['DESYNPUF_ID']
    cols = new_df.columns.values.tolist()    #Make a list of all of the columns in the df
    cols.pop(cols.index('DESYNPUF_ID'))
    new_df = new_df[['DESYNPUF_ID']+cols]    # adjust columns orders that 'DESYNPUF_ID' is the first column
    return new_df

In [173]:
def colname(df):
    """
    Using icd9 codes as columns names.
    """
    return ['DESYNPUF_ID']+[col.split("_")[-1] for col in df.columns.tolist()[1:]] 

In [174]:
def seperate_col(col_lst):
    """
    Identifies the columns that are unique or duplicated by their names.
    """
    duplicated_cols = set()
    unique_cols = set()
    for col in col_lst:
        if col_lst.count(col) == 1 :
            unique_cols.add(col)
        elif col_lst.count(col) > 1:
            duplicated_cols.add(col)
    return (duplicated_cols, unique_cols)
    

In [175]:
def unique_df(df,cols):
    """
    Extract unique columns variables as new data frame.
    """
    
    new_df = df[cols]
    new_df = pd.concat([df[['DESYNPUF_ID']],new_df], axis = 1)
    return new_df

In [310]:
def dup_df(df,col_lst):
    """
    Go through the values of duplicated columns grouped by their column names. 
    If there is 1 in these values, then the value of a new column for this code is 1.
    Concat these columns as a new data frame.
    """
    new_df = df[['DESYNPUF_ID']]
    for col in col_lst:
        new_col = np.zeros(df.shape[0], dtype = np.int16)
        sub_df = df[col]
        idx = []
        for i in range(sub_df.shape[0]):
            if 1 in list(sub_df.iloc[i]):
                idx += [i]
        new_col[idx] = 1
        df2 = pd.DataFrame(new_col, columns=[col+'dum'])
        new_df = pd.concat([new_df, df2], axis=1)
    new_df.pop('0dum')
    return new_df

In [251]:
df_new = df_out_chem.copy()

In [252]:
ICD9_DGNS_cols = [col for col in df_new.columns.tolist() if col[0:6] == 'ICD9_D']
icd9_df = df_dummies(df_new, ICD9_DGNS_cols)

In [253]:
icd9_df.shape

(2302, 2742)

In [317]:
new_col = colname(icd9_df)
icd9_df.columns = new_col

In [318]:
new_col.pop(0)

'DESYNPUF_ID'

In [319]:
dup_diag_cols, uni_diag_cols = seperate_col(new_col)

In [321]:
len(dup_diag_cols), len(uni_diag_cols)

(516, 560)

In [322]:
uni_diag_df = unique_df(icd9_df,uni_diag_cols)

In [323]:
uni_diag_df.shape

(2302, 561)

In [324]:
uni_diag_df.head()

,DESYNPUF_ID,7944,V660,8076,80502,51881,V454,2849,3589,20301,59801,79009,V700,2882,71947,2384,99831,6980,7299,1543,5368,73730,V290,34291,4412,7020,2390,20900,79511,5118,28310,42761,1518,71589,2831,40311,7289,1821,24900,4292,5680,V1505,2881,6827,71597,1410,5950,78071,79501,1562,1948,38612,5733,72691,2822,25040,71948,1599,V815,62133,V6406,5128,5719,V769,1739,E9307,V661,2865,E8792,28260,1610,78729,V441,2148,4779,V1651,4539,11284,38840,78052,9597,2827,2900,23332,73309,6952,5355,938,28809,7837,3669,4911,2692,71596,20274,38651,E8490,28401,20930,V440,5778,07032,E8498,3962,82010,28249,4139,V1007,95901,2839,45119,30928,20501,2631,27541,1838,2558,59970,7859,5970,71500,1572,1482,V436,217,1521,20240,28950,V145,6101,2824,53361,V581,V1209,2848,1522,2812,V5876,5569,2829,4612,V1509,V1000,V7644,20220,6216,V653,1940,2560,20038,2821,7212,538,5935,4536,28311,78961,4871,27903,73679,9130,7871,2814,V598,23874,2884,1630,78965,72210,62989,5959,V1004,53640,V090,7383,71515,7852,4140,6182,20281,44389,V666,4289,V5882,V1329,4940,5758,1469,4258,71967,7831,1598,7851,6103,78763,E9330,7904,2842,V473,4476,1120,79381,V1507,5779,V1003,3019,4142,78050,70715,6262,72239,2689,2329,V8709,58881,28959,E8497,7489,78969,5968,1768,V5864,99813,V198,82100,7942,25541,4169,4730,22809,2820,2813,7891,5921,70714,V0261,34581,56211,7930,28409,V5832,3898,4279,4571,59589,2536,20160,20143,29590,28864,V148,4580,07070,4419,9681,1719,78449,20041,4470,20975,5600,30002,4549,6258,1580,V109,7962,V555,35800,V5841,28319,2828,V816,2753,3454,57410,99779,28804,74769,53649,79099,7932,586,4160,2386,36250,78830,5934,1834,7827,72990,05319,78820,2452,20490,24291,2391,71940,2189,4148,5111,1832,E9300,V676,4264,31401,2143,1750,6113,V6442,7038,5793,2443,4550,2331,99799,25002,37730,38633,78723,0389,20502,2801,20972,5264,79539,61189,7841,3441,2808,52809,2512,9713,7869,1490,23879,V518,V6751,4941,79503,99529,2818,33819,28749,7810,71946,99681,5756,7102,2840,39891,7222,29900,49122,V7646,78442,V1041,V691,51189,1910,28989,5845,5931,20902,38611,V7611,1808,6869,94522,V2509,61179,1713,V670,20050,4239,2307,71509,23691,2722,20412,73028,587,1542,20411,28863,42760,72982,0549,4582,1478,38830,1403,78651,7010,5722,70219,1101,72783,2777,7824,V4509,V1649,37993,78097,2360,28481,V7285,V1079,8056,V5350,V453,30393,V539,20043,56989,28859,20073,56203,1761,V584,5997,319,6259,2113,V5873,3688,7282,70400,1420,3368,5581,2114,36610,3331,2270,71949,V1640,3342,2353,9565,V5830,2411,2462,3510,20014,45189,1648,2763,70901,9998,1737,3568,32712,3577,5649,V184,1569,28953,1707,8054,1769,2440,5121,0413,1639,2989,99859,E8499,V171,2448,72190,V169,78600,6989,V1091,82003,72400,1880,4430,4592,28739,V8402,5881,5847,41512,61801,20287,2823,45981,3562,V4586,1123,53010,1560,56889,78907,1869,3549,61171,55203,6825,7868,5721,V435,4659,42832,V5842,8798,99683,4464,V166,1701,20035,2765,7041,5790,V7263,2332,V5042,4401,29620,5734,28862,3319,70720,40290,48230,4540,4532,E8798,V7282,2851,56213,72999,5181,135,7241,2880,V0752,20286,7292,V1088,24290,4572,20903,V4382,5903,92410,87342,5728,5852,V51,1740,V4981,70724,8439,7917,7534,38630,78340,5848
0,42CDA84C361BDF03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [325]:
uni_diag_df['7944'].value_counts()

0    2301
1       1
Name: 7944, dtype: int64

In [326]:
uni_diag_df['V660'].value_counts()

0    2300
1       2
Name: V660, dtype: int64

In [327]:
%%time
dup_diag_df = dup_df(icd9_df,dup_diag_cols)

CPU times: user 1min 45s, sys: 540 ms, total: 1min 46s
Wall time: 1min 46s


In [328]:
dup_diag_df.shape

(2302, 516)

In [330]:
dup_diag_df.head()

,DESYNPUF_ID,4414dum,7919dum,340dum,41400dum,5939dum,V1641dum,5854dum,1965dum,7931dum,V7269dum,2767dum,V163dum,72252dum,2731dum,V726dum,5920dum,V7612dum,53081dum,4011dum,78096dum,5738dum,V161dum,7830dum,4519dum,1977dum,2409dum,20280dum,5559dum,1912dum,1628dum,V7231dum,33394dum,4240dum,29680dum,2809dum,V707dum,5849dum,1561dum,3320dum,E9479dum,V5831dum,49121dum,1759dum,7892dum,20000dum,60000dum,7295dum,3004dum,2410dum,27651dum,V443dum,2858dum,20288dum,1963dum,1983dum,78951dum,4739dum,V7286dum,71945dum,2312dum,2810dum,1571dum,1619dum,V4581dum,1588dum,1968dum,V662dum,61172dum,28731dum,1727dum,6110dum,73739dum,V4572dum,33829dum,20010dum,7820dum,7936dum,7231dum,27801dum,7856dum,59389dum,78863dum,V5863dum,2888dum,1541dum,7994dum,V5865dum,V427dum,4162dum,2899dum,20300dum,7881dum,28521dum,19882dum,5715dum,73301dum,5533dum,486dum,1985dum,V1052dum,78039dum,V5889dum,V1272dum,1962dum,30000dum,2729dum,V5869dum,28730dum,79589dum,20282dum,3310dum,2841dum,41401dum,E9331dum,V1042dum,V7281dum,3569dum,7821dum,5739dum,56210dum,1519dum,1570dum,1982dum,V580dum,V4282dum,1533dum,71536dum,3558dum,5589dum,25060dum,V4571dum,V141dum,78079dum,V860dum,V1251dum,5305dum,V4573dum,E8788dum,5720dum,V1043dum,725dum,25000dum,1728dum,51889dum,V5812dum,78702dum,20400dum,1972dum,20890dum,V1046dum,5601dum,4400dum,27900dum,60001dum,5183dum,7098dum,40390dum,E9478dum,V153dum,1578dum,78060dum,185dum,7806dum,79093dum,1536dum,40210dum,V5811dum,2662dum,23873dum,7937dum,7804dum,78650dum,1974dum,19889dum,1460dum,7812dum,1961dum,20500dum,1479dum,19881dum,5289dum,79581dum,5789dum,V4502dum,4556dum,2761dum,V5866dum,V6700dum,3384dum,5180dum,78057dum,4293dum,9961dum,7850dum,79399dum,4299dum,20283dum,1969dum,7862dum,34590dum,28522dum,78605dum,7140dum,1622dum,6930dum,V4511dum,V1301dum,V8801dum,V140dum,6959dum,78930dum,79902dum,78061dum,79431dum,78901dum,1505dum,23871dum,42781dum,V4577dum,1889dum,4280dum,78909dum,7213dum,7802dum,1624dum,6822dum,7921dum,179dum,1888dum,5119dum,52801dum,7873dum,V1009dum,40391dum,5932dum,V0481dum,7906dum,V4589dum,72981dum,42789dum,2750dum,1550dum,V142dum,V1582dum,2720dum,3559dum,V6759dum,78321dum,20510dum,1984dum,5856dum,99883dum,2559dum,6826dum,4599dum,2850dum,78720dum,1987dum,V1089dum,27800dum,V168dum,2768dum,1535dum,78906dum,2639dum,51181dum,V679dum,28489dum,1529dum,5723dum,99674dum,V180dum,2724dum,1966dum,49320dum,V1051dum,V762dum,V462dum,45111dum,32723dum,V1087dum,23875dum,1840dum,4371dum,20203dum,72251dum,3383dum,71699dum,V6709dum,1729dum,71690dum,20018dum,V7260dum,78701dum,5531dum,78099dum,53019dum,20208dum,V1254dum,3572dum,1532dum,6238dum,7100dum,1419dum,20921dum,75310dum,V711dum,1950dum,2875dum,7242dum,4538dum,5990dum,55320dum,36960dum,7910dum,71941dum,1978dum,2859dum,V138dum,2874dum,2830dum,591dum,7847dum,45989dum,V7283dum,V5867dum,1744dum,78939dum,1990dum,1551dum,2769dum,45341dum,5609dum,78903dum,3051dum,V5861dum,7244dum,V103dum,27650dum,V1005dum,V861dum,1520dum,42731dum,78652dum,73300dum,1746dum,1534dum,V4501dum,47819dum,1629dum,V1259dum,79092dum,6102dum,V420dum,20200dum,78799dum,V1279dum,29410dum,V5881dum,2330dum,V725dum,1970dum,34690dum,5859dum,1976dum,V1006dum,99520dum,2733dum,23872dum,V066dum,27542dum,2592dum,V8741dum,462dum,45340dum,71590dum,78659dum,1623dum,78904dum,41519dum,78900dum,79389dum,V123dum,1809dum,V1589dum,311dum,V5883dum,3576dum,1973dum,59582dum,V159dum,1745dum,1539dum,515dum,4019dum,1589dum,5110dum,1625dum,6279dum,1510dum,78843dum,1743dum,72887dum,2800dum,20302dum,V160dum,78841dum,1820dum,V8401dum,490dum,7099dum,57420dum,V4365dum,7224dum,07054dum,V1261dum,7964dum,1906dum,V072dum,78933dum,1537dum,5853dum,2469dum,28803dum,4928dum,V6549dum,1579dum,V1011dum,496dum,2819dum,V4989dum,28850dum,V4364dum,412dum,1828dum,20410dum,53550dum,1540dum,07030dum,2449dum,7866dum,1971dum,7840dum,28800dum,7291dum,2948dum,2749dum,V672dum,78791dum,V1082dum,2811dum,78703dum,1975dum,20040dum,V7284dum,1741dum,1830dum,1749dum,1742dum,28860dum,4439dum,56200dum,1960dum,1890dum,5772dum,28529dum,V1588dum,V1508dum,37230dum,20979dum,56400dum,1509dum,V5862dum,

In [331]:
dup_diag_df['4414dum'].value_counts()

0    2299
1       3
Name: 4414dum, dtype: int64

In [332]:
dup_diag_df['5939dum'].value_counts()

0    2285
1      17
Name: 5939dum, dtype: int64

In [374]:
dup_diag_df.pop('DESYNPUF_ID')

0       42CDA84C361BDF03
1       44DA5770803B6370
2       486BE10685E7A377
3       519B21BB4C4DD253
4       55C76CA11E982610
              ...       
2297    FCF705B91C2E9898
2298    FE1312EF5BCDE363
2299    FE209AAB0CCAF602
2300    FF959803021EA3B5
2301    FFC32B2413B5E345
Name: DESYNPUF_ID, Length: 2302, dtype: object

In [375]:
df_ml_diag = pd.concat([uni_df,dup_diag_df], axis = 1)

In [376]:
df_ml_diag.shape

((2360, 1076), (2302, 1076))

In [337]:
df_ml_diag.head()

,DESYNPUF_ID,7944,V660,8076,80502,51881,V454,2849,3589,20301,59801,79009,V700,2882,71947,2384,99831,6980,7299,1543,5368,73730,V290,34291,4412,7020,2390,20900,79511,5118,28310,42761,1518,71589,2831,40311,7289,1821,24900,4292,5680,V1505,2881,6827,71597,1410,5950,78071,79501,1562,1948,38612,5733,72691,2822,25040,71948,1599,V815,62133,V6406,5128,5719,V769,1739,E9307,V661,2865,E8792,28260,1610,78729,V441,2148,4779,V1651,4539,11284,38840,78052,9597,2827,2900,23332,73309,6952,5355,938,28809,7837,3669,4911,2692,71596,20274,38651,E8490,28401,20930,V440,5778,07032,E8498,3962,82010,28249,4139,V1007,95901,2839,45119,30928,20501,2631,27541,1838,2558,59970,7859,5970,71500,1572,1482,V436,217,1521,20240,28950,V145,6101,2824,53361,V581,V1209,2848,1522,2812,V5876,5569,2829,4612,V1509,V1000,V7644,20220,6216,V653,1940,2560,20038,2821,7212,538,5935,4536,28311,78961,4871,27903,73679,9130,7871,2814,V598,23874,2884,1630,78965,72210,62989,5959,V1004,53640,V090,7383,71515,7852,4140,6182,20281,44389,V666,4289,V5882,V1329,4940,5758,1469,4258,71967,7831,1598,7851,6103,78763,E9330,7904,2842,V473,4476,1120,79381,V1507,5779,V1003,3019,4142,78050,70715,6262,72239,2689,2329,V8709,58881,28959,E8497,7489,78969,5968,1768,V5864,99813,V198,82100,7942,25541,4169,4730,22809,2820,2813,7891,5921,70714,V0261,34581,56211,7930,28409,V5832,3898,4279,4571,59589,2536,20160,20143,29590,28864,V148,4580,07070,4419,9681,1719,78449,20041,4470,20975,5600,30002,4549,6258,1580,V109,7962,V555,35800,V5841,28319,2828,V816,2753,3454,57410,99779,28804,74769,53649,79099,7932,586,4160,2386,36250,78830,5934,1834,7827,72990,05319,78820,2452,20490,24291,2391,71940,2189,4148,5111,1832,E9300,V676,4264,31401,2143,1750,6113,V6442,7038,5793,2443,4550,2331,99799,25002,37730,38633,78723,0389,20502,2801,20972,5264,79539,61189,7841,3441,2808,52809,2512,9713,7869,1490,23879,V518,V6751,4941,79503,99529,2818,33819,28749,7810,71946,99681,5756,7102,2840,39891,7222,29900,49122,V7646,78442,V1041,V691,51189,1910,28989,5845,5931,20902,38611,V7611,1808,6869,94522,V2509,61179,1713,V670,20050,4239,2307,71509,23691,2722,20412,73028,587,1542,20411,28863,42760,72982,0549,4582,1478,38830,1403,78651,7010,5722,70219,1101,72783,2777,7824,V4509,V1649,37993,78097,2360,28481,V7285,V1079,8056,V5350,V453,30393,V539,20043,56989,28859,20073,56203,1761,V584,5997,319,6259,2113,V5873,3688,7282,70400,1420,3368,5581,2114,36610,3331,2270,71949,V1640,3342,2353,9565,V5830,2411,2462,3510,20014,45189,1648,2763,70901,9998,1737,3568,32712,3577,5649,V184,1569,28953,1707,8054,1769,2440,5121,0413,1639,2989,99859,E8499,V171,2448,72190,V169,78600,6989,V1091,82003,72400,1880,4430,4592,28739,V8402,5881,5847,41512,61801,20287,2823,45981,3562,V4586,1123,53010,1560,56889,78907,1869,3549,61171,55203,6825,7868,5721,V435,4659,42832,V5842,8798,99683,4464,V166,1701,20035,2765,7041,5790,V7263,2332,V5042,4401,29620,5734,28862,3319,70720,40290,48230,4540,4532,E8798,V7282,2851,56213,72999,5181,135,7241,2880,V0752,20286,7292,V1088,24290,4572,20903,V4382,5903,92410,87342,5728,5852,V51,1740,V4981,70724,8439,7917,7534,38630,78340,5848,DESYNPUF_ID,4414dum,7919dum,340dum,41400dum,5939dum,V1641dum,5854dum,1965dum,7931dum,V7269dum,2767dum,V163dum,72252dum,2731dum,V726dum,5920dum,V7612dum,53081dum,4011dum,78096dum,5738dum,V161dum,7830dum,4519dum,1977dum,2409dum,20280dum,5559dum,1912dum,1628dum,V7231dum,33394dum,4240dum,29680dum,2809dum,V707dum,5849dum,1561dum,3320dum,E9479dum,V5831dum,49121dum,1759dum,7892dum,20000dum,60000dum,7295dum,3004dum,2410dum,27651dum,V443dum,2858dum,20288dum,1963dum,1983dum,78951dum,4739dum,V7286dum,71945dum,2312dum,2810dum,1571dum,1619dum,V4581dum,1588dum,1968dum,V662dum,61172dum,28731dum,1727dum,6110dum,73739dum,V4572dum,33829dum,20010dum,7820dum,7936dum,7231dum,27801dum,7856dum,59389dum,78863dum,V5863dum,2888dum,1541dum,7994dum,V5865dum,V427dum,4162dum,2899dum,20300dum,7881dum,28521dum,19882dum,5715dum,73301dum,5533dum,486dum,1985dum,V1052dum,78039dum,V5889dum,V1272dum,1962dum,30000dum,2729dum,V5869dum,28730dum,79589dum,20282dum,3310dum,

### ANOTHER WAY
Get all the unique values  
Check row by row, if have values, that feature = 1  

In [54]:
def uni_col(df, cols):
    """
    Collect all unique values for icd diagnosis or HCPCS codes.
    Return a set with these unique values.
    """
    uniqe_set = set()
    for col in cols:
        new_set = df[col].unique().tolist()
        uniqe_set.update(new_set)
    return uniqe_set

In [55]:
def new_dic(df,dic,i):
    """
    Create a dictionary, keys are unique codes. 
    Extract one row instance and check the values.
    If there is 1 in the values of one code from the data, the values of this key is [1], or is [0].
    Return the dictionary.
    """
    values = list(df.iloc[i])
    for val in values:
        if val !='0':
            dic[val] = [1]
    return dic

In [110]:
def onehot_df(df,id_name, cols):

    feature = list(uni_col(df, cols))
    dic_int = {}
    for f in feature:
        dic_int[f] = [0] 
    dic_0 = dic_int
    dic_0[id_name] = [df[id_name].iloc[0]]
    df0 = df.loc[:,cols]
    dic_0 = new_dic(df0,dic_0,0)
    new_df = pd.DataFrame.from_dict(dic_0)
    x = df.shape[0]
    for i in range(1,x):
        dic = dic_int
        dic = new_dic(df0,dic,i)
        dic[id_name] = [df[id_name].iloc[i]]
        df2 = pd.DataFrame.from_dict(dic)
        new_df = pd.concat([new_df,df2])
    return new_df   

In [111]:
%%time
dummy_df = onehot_df(icd9_df,'DESYNPUF_ID', ICD9_DGNS_cols)

CPU times: user 2min 9s, sys: 8.16 s, total: 2min 17s
Wall time: 2min 17s


In [116]:
dummy_df.pop('0')


0    0
0    0
0    0
0    0
0    0
    ..
0    0
0    0
0    0
0    0
0    0
Name: 0, Length: 2302, dtype: int64

In [117]:
dummy_df.shape

(2302, 1076)

### *Onehot_code with ICD_PRECEDURE_CODE*

In [338]:
ICD9_PREC_cols = [col for col in df_new.columns.tolist() if col[0:6] == 'ICD9_P']

In [339]:
prec_df = df_dummies(df_new, ICD9_PREC_cols)
new_col = colname(prec_df)
prec_df.columns = new_col
new_col.pop(0)
dup_prec_cols, uni_prec_cols = seperate_col(new_col)

In [340]:
len(dup_prec_cols), len(uni_prec_cols)

(3, 17)

In [341]:
prec_unique = unique_df(prec_df,unique_cols)
prec_dupli = dup_df(prec_df,duplicated_cols)

In [342]:
prec_unique.shape,prec_dupli.shape

((2302, 18), (2302, 3))

In [343]:
l1 = prec_unique.DESYNPUF_ID.values.tolist()

In [344]:
l2 = prec_dupli.DESYNPUF_ID.values.tolist()

In [345]:
l1 == l2

True

In [346]:
prec_unique.head()

,DESYNPUF_ID,7852,5762,2720,2768,2859,3491,0419,60000,V1043,2869,9671,1749,28959,58881,1975,5794,5491
0,42CDA84C361BDF03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,44DA5770803B6370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,486BE10685E7A377,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,519B21BB4C4DD253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,55C76CA11E982610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [347]:
prec_unique['7852'].value_counts()

0    2300
1       2
Name: 7852, dtype: int64

In [348]:
prec_unique['V1043'].value_counts()

0    2301
1       1
Name: V1043, dtype: int64

In [349]:
prec_dupli.head()

,DESYNPUF_ID,V5812dum,V5811dum
0,42CDA84C361BDF03,0,0
1,44DA5770803B6370,0,0
2,486BE10685E7A377,0,0
3,519B21BB4C4DD253,0,0
4,55C76CA11E982610,0,0


In [350]:
prec_dupli['V5812dum'].value_counts()

0    2300
1       2
Name: V5812dum, dtype: int64

In [351]:
prec_dupli['V5811dum'].value_counts()

0    2298
1       4
Name: V5811dum, dtype: int64

In [371]:
prec_dupli.pop('DESYNPUF_ID')

0       42CDA84C361BDF03
1       44DA5770803B6370
2       486BE10685E7A377
3       519B21BB4C4DD253
4       55C76CA11E982610
              ...       
2297    FCF705B91C2E9898
2298    FE1312EF5BCDE363
2299    FE209AAB0CCAF602
2300    FF959803021EA3B5
2301    FFC32B2413B5E345
Name: DESYNPUF_ID, Length: 2302, dtype: object

In [372]:
df_ml_prec = pd.concat([prec_unique, prec_dupli], axis = 1)

In [353]:
df_ml_prec_2 = pd.merge(prec_unique, prec_dupli, on = 'DESYNPUF_ID', how = "inner")

In [373]:
df_ml_prec.shape, df_ml_prec_2.shape

((2302, 20), (2360, 20))

### Onehot_code with HCPCS_CODE

In [355]:
ICD9_HCPCS_cols = [col for col in df_new.columns.tolist() if col[0:5] == 'HCPCS']

In [356]:
hcpcs_df = df_dummies(df_new, ICD9_HCPCS_cols)
hcpcs_col = colname(hcpcs_df)
hcpcs_df.columns = hcpcs_col
hcpcs_col.pop(0)
duplicated_hc_cols, unique_hc_cols = seperate_col(hcpcs_col)

In [357]:
len(duplicated_hc_cols), len(unique_hc_cols)

(906, 599)

In [358]:
uni_hc_df = unique_df(hcpcs_df,unique_hc_cols)
#hc_dupli = dup_df(hcpcs_df,duplicated_hc_cols)

In [359]:
%%time
dup_hc_df = dup_df(hcpcs_df,duplicated_hc_cols)

CPU times: user 3min 7s, sys: 1.65 s, total: 3min 8s
Wall time: 3min 8s


In [360]:
uni_hc_df.shape

(2302, 600)

In [361]:
dup_hc_df.shape

(2302, 906)

In [362]:
uni_hc_df.head()

,DESYNPUF_ID,G0390,68761,36593,52234,43260,J2300,73701,P9019,99217,J0702,Q4098,G0383,P9603,33249,90585,J9280,85270,99406,75825,28810,86336,15360,77790,50387,52351,28310,J0585,87176,G0289,46945,36005,87496,L8699,96542,A6251,00920,J1840,C1898,81005,27788,72146,84445,A4649,C1773,A9567,J1740,84305,37206,17311,J1120,82731,Q9958,82365,J0692,87799,87339,70360,83835,95934,30905,J3590,86694,C1779,74475,J7511,49650,90804,82941,11624,82375,A9560,77013,0197T,63685,29825,J2778,20225,51797,28090,82670,G0129,J0895,95861,51040,45331,78585,75966,22520,74410,67950,12034,86674,19340,11602,73070,49081,21029,85380,A4450,36147,11606,52317,76350,70160,35476,93798,51610,75964,13122,93721,23700,55520,22899,64999,74360,12053,29823,74305,33215,A0428,87390,67036,90472,52260,C1771,99218,J9090,74400,73010,15220,15120,12051,20220,A4363,72195,73721,54161,31528,82360,22521,19101,55040,97016,64626,35473,99234,67311,J9098,70250,00400,86664,49652,L3265,95904,91105,76880,J2180,36821,87076,57421,82024,75665,A4641,83876,73040,J7517,32552,83912,76999,87272,J3410,52310,14040,67025,G0380,73565,76080,43259,86162,72265,37204,10061,66986,21930,Q0164,G0251,36830,84702,63056,70548,93308,99235,66250,87497,84146,J1110,C1885,45334,94750,88360,A6452,42299,73700,49521,99205,96450,A6212,64718,87276,72131,74220,86367,35474,20690,74245,52315,77401,12032,93976,C1731,A9535,C1883,15004,V2785,82668,82164,J2993,A0427,33240,38500,J1300,31540,84207,20550,A6260,80170,87427,29530,36200,87798,J9290,26210,75992,73660,J3243,87430,A4223,86940,77370,86703,A6196,36575,86701,72240,75685,33222,72295,21480,19307,84152,Q1003,70549,74250,C2630,87332,76001,47399,J9268,33216,20206,83013,72275,11422,J7507,70480,86788,A6203,72156,Q9968,J2353,P9032,J1569,A9503,29105,87899,73223,82542,J2597,93652,A6206,L4386,P9612,73500,42415,88161,85540,86003,70490,52601,47505,13121,J0135,99149,93351,87803,82950,86171,J9096,76377,86698,38525,G0332,95811,J2720,L8606,C2628,83909,49572,J7340,A4218,99204,31526,J3250,93571,26531,C8902,74230,19330,41108,10140,29807,J0882,19291,95819,84432,87101,74240,97760,77321,73222,95970,89055,20605,84133,J0595,52352,69436,41112,83901,86663,87210,17250,87536,86886,38206,87275,01999,77778,A4301,28299,94002,P9048,86612,67840,P9058,J2310,99292,J9291,95900,00532,54530,74300,A6443,20680,Q4101,99195,80102,41899,12014,95972,36581,J9050,87591,93925,97605,45999,67040,14000,C1882,70543,A4346,90656,88325,J2260,25607,L0120,J1630,A6453,Q0081,50393,V2788,J1327,G0204,97032,75724,76776,31575,C1897,11765,P3000,99499,86666,36590,29126,36478,00914,64470,99150,20999,93922,58120,11402,J9182,66761,82810,51728,71270,J8610,76873,72128,80200,45340,29075,43247,P9045,93224,64491,80195,C2625,J2360,49325,51736,63047,76536,33207,36140,11644,P9031,64405,11642,64495,52005,45335,12013,73000,92250,27093,54416,87209,J1459,36522,G0104,72052,22612,67005,83900,71035,76775,11603,80500,A6228,76950,23655,86611,77406,Q9962,46922,A6402,G0340,31525,28465,37766,C1724,C1786,87149,83873,93797,43888,26116,28092,31622,67314,86022,80202,82747,J7611,65850,72190,70492,11626,25076,36410,75898,J0835,55876,J9207,00529,86343,82088,C8929,30630,49329,43255,84154,0147T,C9399,46080,25310,78812,75650,75790,C1714,19110,49440,97018,23420,52001,63655,49423,86665,23515,88142,93508,J0559,J1790,J1568,49422,67042,44361,86617,85348,83516,85002,64402,31536,37203,15840,82652,66030,82800,87185,19102,88302,A6021,83630,49180,J0743,83896,27618,43264,23120,67917,88333,C1815,J9040,A0429,97112,70470,93923,26951,49520,29425,75710,C9242,73702,C1717,C1764,93542,L8614,15822,64475,85032,74246,74320,A4300,J9017,82274,A4216,26765,97113,J2800,92981,90700,J9303,86807,94620,43251,26440,76872
0,42CDA84C361BDF03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [364]:
dup_hc_df.head()

,DESYNPUF_ID,38792dum,87086dum,73560dum,84153dum,84479dum,C1769dum,93012dum,J3490dum,C1874dum,84075dum,88173dum,Q9957dum,99211dum,99284dum,G0009dum,72148dum,86141dum,87177dum,84478dum,86140dum,83605dum,J9360dum,97150dum,82607dum,J3130dum,82247dum,52353dum,43450dum,80197dum,C1726dum,31255dum,J2765dum,72158dum,86360dum,84480dum,62310dum,77051dum,86039dum,29540dum,82552dum,83540dum,51701dum,36215dum,73090dum,36556dum,J2357dum,94642dum,57288dum,64640dum,64450dum,J1441dum,71020dum,99285dum,83519dum,87077dum,77032dum,50394dum,J9263dum,J9045dum,G0123dum,82232dum,87147dum,C1776dum,93005dum,88312dum,43245dum,82140dum,51720dum,J9375dum,97039dum,88185dum,80164dum,G0105dum,72220dum,86592dum,J2780dum,J1100dum,82746dum,73630dum,J9035dum,J0885dum,72191dum,88112dum,J7502dum,J1160dum,85025dum,P9037dum,97606dum,97033dum,J0640dum,77014dum,78195dum,99291dum,G0290dum,78472dum,72193dum,88309dum,P9035dum,J9170dum,J9208dum,A9552dum,88311dum,76000dum,76705dum,A9577dum,C1887dum,77416dum,99283dum,87046dum,28285dum,96368dum,90766dum,32405dum,99219dum,14060dum,96415dum,Q2024dum,J3430dum,A9558dum,77002dum,45385dum,99203dum,J1572dum,82575dum,J9033dum,J9031dum,90714dum,J9025dum,A9562dum,97116dum,97001dum,78480dum,G0202dum,Q0166dum,G0260dum,43269dum,Q9967dum,82310dum,J9350dum,74175dum,47562dum,75960dum,75726dum,60500dum,75962dum,92980dum,90761dum,86803dum,P9040dum,43760dum,84443dum,86900dum,78588dum,77414dum,11044dum,G0393dum,J9100dum,G0306dum,82465dum,G0382dum,82565dum,97535dum,95810dum,J7506dum,J0150dum,15340dum,87400dum,94760dum,82248dum,88305dum,93543dum,78452dum,J1956dum,11041dum,C1876dum,74000dum,93971dum,J1745dum,J8499dum,94762dum,99144dum,96365dum,72072dum,77331dum,78306dum,J0878dum,62350dum,33208dum,73120dum,66984dum,J9178dum,J7042dum,96366dum,88307dum,84402dum,93651dum,85004dum,62362dum,81002dum,80047dum,J2916dum,45330dum,86885dum,97022dum,77334dum,20600dum,85576dum,74183dum,78223dum,85027dum,G0008dum,J1642dum,96376dum,J2469dum,88271dum,C1780dum,83010dum,83550dum,J7060dum,71101dum,36600dum,99214dum,72132dum,G0431dum,82330dum,97035dum,70498dum,36415dum,J1260dum,77280dum,82550dum,J3480dum,97124dum,87641dum,86316dum,86255dum,52332dum,84166dum,94660dum,25605dum,36584dum,64721dum,36561dum,80051dum,93621dum,83735dum,J2543dum,82270dum,75827dum,72020dum,A6258dum,L1830dum,C1750dum,71010dum,80048dum,J2323dum,J2250dum,49585dum,11042dum,76376dum,96416dum,83090dum,J9390dum,85246dum,49568dum,C1781dum,33233dum,J3488dum,J1250dum,84403dum,J1440dum,10022dum,45378dum,45383dum,C8928dum,20552dum,92960dum,36569dum,36430dum,87220dum,87116dum,82272dum,99201dum,88264dum,86644dum,J8540dum,36245dum,G0121dum,74022dum,82040dum,77003dum,43268dum,64622dum,82945dum,36514dum,88304dum,J0583dum,00810dum,77290dum,88262dum,76380dum,74170dum,J9181dum,82306dum,96361dum,76830dum,36246dum,A9539dum,85018dum,62311dum,76700dum,P9016dum,86920dum,85049dum,85384dum,J1626dum,74330dum,J1030dum,82728dum,64493dum,J1650dum,J2930dum,J7120dum,86618dum,J9293dum,70553dum,J2405dum,87184dum,80076dum,85007dum,J0881dum,86304dum,36416dum,45384dum,73030dum,73130dum,J2275dum,92526dum,85245dum,92135dum,73562dum,J9091dum,58661dum,93613dum,36591dum,Q0180dum,83970dum,82055dum,19103dum,G0278dum,C1725dum,12052dum,77001dum,62290dum,45381dum,82435dum,52000dum,29875dum,94360dum,29876dum,74020dum,93556dum,95990dum,83874dum,76937dum,J9264dum,J0780dum,52204dum,87230dum,36000dum,77421dum,90715dum,J1245dum,93225dum,67041dum,75630dum,C1300dum,J9185dum,87804dum,29826dum,99145dum,J3010dum,J3470dum,J2505dum,38510dum,64623dum,84450dum,75820dum,80162dum,J1335dum,J1756dum,73564dum,12011dum,86945dum,80069dum,43242dum,86038dum,96402dum,90999dum,C2617dum,73590dum,84439dum,80156dum,94060dum,J9265dum,83690dum,71090dum,87340dum,30901dum,J2710dum,70544dum,J9093dum,88184dum,73510dum,81001dum,90732dum,86430dum,87880dum,96523dum,82043dum,95903dum,99213dum,A9579dum,84165dum,J9305dum,V2632dum,C1879dum,22523dum,70450dum,77418dum,P9041dum,76098dum,83883dum,93544dum,88175dum,G0379dum,99212dum,51741dum,80158dum,J2430dum,85610dum,85651dum,93880dum,

In [366]:
dup_hc_df['84479dum'].value_counts()

0    2296
1       6
Name: 84479dum, dtype: int64

In [367]:
dup_hc_df.pop('DESYNPUF_ID')

0       42CDA84C361BDF03
1       44DA5770803B6370
2       486BE10685E7A377
3       519B21BB4C4DD253
4       55C76CA11E982610
              ...       
2297    FCF705B91C2E9898
2298    FE1312EF5BCDE363
2299    FE209AAB0CCAF602
2300    FF959803021EA3B5
2301    FFC32B2413B5E345
Name: DESYNPUF_ID, Length: 2302, dtype: object

In [368]:
df_ml_hcpcs = pd.concat([uni_hc_df, dup_hc_df], axis =1)

In [369]:
df_ml_hcpcs.shape

(2302, 1505)

In [370]:
df_ml_hcpcs.head()

,DESYNPUF_ID,G0390,68761,36593,52234,43260,J2300,73701,P9019,99217,J0702,Q4098,G0383,P9603,33249,90585,J9280,85270,99406,75825,28810,86336,15360,77790,50387,52351,28310,J0585,87176,G0289,46945,36005,87496,L8699,96542,A6251,00920,J1840,C1898,81005,27788,72146,84445,A4649,C1773,A9567,J1740,84305,37206,17311,J1120,82731,Q9958,82365,J0692,87799,87339,70360,83835,95934,30905,J3590,86694,C1779,74475,J7511,49650,90804,82941,11624,82375,A9560,77013,0197T,63685,29825,J2778,20225,51797,28090,82670,G0129,J0895,95861,51040,45331,78585,75966,22520,74410,67950,12034,86674,19340,11602,73070,49081,21029,85380,A4450,36147,11606,52317,76350,70160,35476,93798,51610,75964,13122,93721,23700,55520,22899,64999,74360,12053,29823,74305,33215,A0428,87390,67036,90472,52260,C1771,99218,J9090,74400,73010,15220,15120,12051,20220,A4363,72195,73721,54161,31528,82360,22521,19101,55040,97016,64626,35473,99234,67311,J9098,70250,00400,86664,49652,L3265,95904,91105,76880,J2180,36821,87076,57421,82024,75665,A4641,83876,73040,J7517,32552,83912,76999,87272,J3410,52310,14040,67025,G0380,73565,76080,43259,86162,72265,37204,10061,66986,21930,Q0164,G0251,36830,84702,63056,70548,93308,99235,66250,87497,84146,J1110,C1885,45334,94750,88360,A6452,42299,73700,49521,99205,96450,A6212,64718,87276,72131,74220,86367,35474,20690,74245,52315,77401,12032,93976,C1731,A9535,C1883,15004,V2785,82668,82164,J2993,A0427,33240,38500,J1300,31540,84207,20550,A6260,80170,87427,29530,36200,87798,J9290,26210,75992,73660,J3243,87430,A4223,86940,77370,86703,A6196,36575,86701,72240,75685,33222,72295,21480,19307,84152,Q1003,70549,74250,C2630,87332,76001,47399,J9268,33216,20206,83013,72275,11422,J7507,70480,86788,A6203,72156,Q9968,J2353,P9032,J1569,A9503,29105,87899,73223,82542,J2597,93652,A6206,L4386,P9612,73500,42415,88161,85540,86003,70490,52601,47505,13121,J0135,99149,93351,87803,82950,86171,J9096,76377,86698,38525,G0332,95811,J2720,L8606,C2628,83909,49572,J7340,A4218,99204,31526,J3250,93571,26531,C8902,74230,19330,41108,10140,29807,J0882,19291,95819,84432,87101,74240,97760,77321,73222,95970,89055,20605,84133,J0595,52352,69436,41112,83901,86663,87210,17250,87536,86886,38206,87275,01999,77778,A4301,28299,94002,P9048,86612,67840,P9058,J2310,99292,J9291,95900,00532,54530,74300,A6443,20680,Q4101,99195,80102,41899,12014,95972,36581,J9050,87591,93925,97605,45999,67040,14000,C1882,70543,A4346,90656,88325,J2260,25607,L0120,J1630,A6453,Q0081,50393,V2788,J1327,G0204,97032,75724,76776,31575,C1897,11765,P3000,99499,86666,36590,29126,36478,00914,64470,99150,20999,93922,58120,11402,J9182,66761,82810,51728,71270,J8610,76873,72128,80200,45340,29075,43247,P9045,93224,64491,80195,C2625,J2360,49325,51736,63047,76536,33207,36140,11644,P9031,64405,11642,64495,52005,45335,12013,73000,92250,27093,54416,87209,J1459,36522,G0104,72052,22612,67005,83900,71035,76775,11603,80500,A6228,76950,23655,86611,77406,Q9962,46922,A6402,G0340,31525,28465,37766,C1724,C1786,87149,83873,93797,43888,26116,28092,31622,67314,86022,80202,82747,J7611,65850,72190,70492,11626,25076,36410,75898,J0835,55876,J9207,00529,86343,82088,C8929,30630,49329,43255,84154,0147T,C9399,46080,25310,78812,75650,75790,C1714,19110,49440,97018,23420,52001,63655,49423,86665,23515,88142,93508,J0559,J1790,J1568,49422,67042,44361,86617,85348,83516,85002,64402,31536,37203,15840,82652,66030,82800,87185,19102,88302,A6021,83630,49180,J0743,83896,27618,43264,23120,67917,88333,C1815,J9040,A0429,97112,70470,93923,26951,49520,29425,75710,C9242,73702,C1717,C1764,93542,L8614,15822,64475,85032,74246,74320,A4300,J9017,82274,A4216,26765,97113,J2800,92981,90700,J9303,86807,94620,43251,26440,76872,38792dum,87086dum,73560dum,84153dum,84479dum,C1769dum,93012dum,J3490dum,C1874dum,84075dum,88173dum,Q9957dum,99211dum,99284dum,G0009dum,72148dum,86141dum,87177dum,84478dum,86140dum,83605dum,J9360dum,97150dum,82607dum,J3130dum,82247dum,52353dum,43450dum,80197dum,C1726dum,31255dum,J2765dum,72158dum,86360dum,84480dum,62310dum,77051dum,86039dum,29540dum,82552dum,83540dum,51701dum,36215dum,73090d

In [ ]:
# concat data together: df_ml_diag, df_ml_prec, df_ml_hcpcs

In [379]:
df_ml_prec.pop('DESYNPUF_ID')

0       42CDA84C361BDF03
1       44DA5770803B6370
2       486BE10685E7A377
3       519B21BB4C4DD253
4       55C76CA11E982610
              ...       
2297    FCF705B91C2E9898
2298    FE1312EF5BCDE363
2299    FE209AAB0CCAF602
2300    FF959803021EA3B5
2301    FFC32B2413B5E345
Name: DESYNPUF_ID, Length: 2302, dtype: object

In [380]:
df_ml_hcpcs.pop('DESYNPUF_ID')

0       42CDA84C361BDF03
1       44DA5770803B6370
2       486BE10685E7A377
3       519B21BB4C4DD253
4       55C76CA11E982610
              ...       
2297    FCF705B91C2E9898
2298    FE1312EF5BCDE363
2299    FE209AAB0CCAF602
2300    FF959803021EA3B5
2301    FFC32B2413B5E345
Name: DESYNPUF_ID, Length: 2302, dtype: object

In [383]:
df_ml_diag.shape, df_ml_prec.shape, df_ml_hcpcs.shape

((2302, 1076), (2302, 19), (2302, 1504))

In [381]:
df_ml = pd.concat([df_ml_diag, df_ml_prec, df_ml_hcpcs],axis = 1)

In [384]:
df_ml.shape

(2302, 2599)

In [385]:
df_ml.head()

DESYNPUF_ID  7944  V660  8076  80502  51881  V454  2849  3589  20301  \
0  42CDA84C361BDF03     0     0     0      0      0     0     0     0      0   
1  44DA5770803B6370     0     0     0      0      0     0     0     0      0   
2  486BE10685E7A377     0     0     0      0      0     0     0     0      0   
3  519B21BB4C4DD253     0     0     0      0      0     0     0     0      0   
4  55C76CA11E982610     0     0     0      0      0     0     0     0      0   

   59801  79009  V700  2882  71947  2384  99831  6980  7299  1543  5368  \
0      0      0     0     0      0     0      0     0     0     0     0   
1      0      0     0     0      0     0      0     0     0     0     0   
2      0      0     0     0      0     0      0     0     0     0     0   
3      0      0     0     0      0     0      0     0     0     0     0   
4      0      0     0     0      0     0      0     0     0     0     0   

   73730  V290  34291  4412  7020  2390  20900  79511  5118  28310  42761  \
0      0     0      0     0     0     0      0      0     0      0      0   
1      0     0      0     0     0     0      0      0     0      0      0   
2      0     0      0     0     0     0      0      0     0      0      0   
3      0     0      0     0     0     0      0      0     0      0      0   
4      0     0      0     0     0     0      0      0     0      0      0   

   1518  71589  2831  40311  7289  1821  24900  4292  5680  V1505  2881  6827  \
0     0      0     0      0     0     0      0     0     0      0     0     0   
1     0      0     0      0     0     0      0     0     0      0     0     0   
2     0      0     0      0     0     0      0     0     0      0     0     0   
3     0      0     0      0     0     0      0     0     0      0     0     0   
4     0      0     0      0     0     0      0     0     0      0     0     0   

   71597  1410  5950  78071  79501  1562  1948  38612  5733  72691  2822  \
0      0     0     0      0      0     0     0      0     0      0     0   
1      0     0     0      0      0     0     0      0     0      0     0   
2      0     0     0      0      0     0     0      0     0      0     0   
3      0     0     0      0      0     0     0      0     0      0     0   
4      0     0     0      0      0     0     0      0     0      0     0   

   25040  71948  1599  V815  62133  V6406  5128  5719  V769  1739  E9307  \
0      0      0     0     0      0      0     0     0     0     0      0   
1      0      0     0     0      0      0     0     0     0     0      0   
2      0      0     0     0      0      0     0     0     0     0      0   
3      0      0     0     0      0      0     0     0     0     0      0   
4      0      0     0     0      0      0     0     0     0     0      0   

   V661  2865  E8792  28260  1610  78729  V441  2148  4779  V1651  4539  \
0     0     0      0      0     0      0     0     0     0      0     0   
1     0     0      0      0     0      0     0     0     0      0     0   
2     0     0      0      0     0      0     0     0     0      0     0   
3     0     0      0      0     0      0     0     0     0      0     1   
4     0     0      0      0     0      0     0     0     0      0     0   

   11284  38840  78052  9597  2827  2900  23332  73309  6952  5355  938  \
0      0      0      0     0     0     0      0      0     0     0    0   
1      0      0      0     0     0     0      0      0     0     0    0   
2      0      0      0     0     0     0      0      0     0     0    0   
3      0      0      0     0     0     0      0      0     0     0    0   
4      0      0      0     0     0     0      0      0     0     0    0   

   28809  7837  3669  4911  2692  71596  20274  38651  E8490  28401  20930  \
0      0     0     0     0     0      0      0      0      0      0      0   
1      0     0     0     0     0      0      0      0      0      0      0   
2      0     0     0     0     0      0      0      0      0      0      0  

In [386]:
df_ml.to_csv("/Users/jill/Downloads/2020_Project/EDA/concat_data/onehot_df.csv")